## ChromaDB Vector Database Deployment on AWS
###### Create the vector database and deploy it on AWS

In [1]:
from chromadb.utils import embedding_functions

# Set embedding model
chromadb_embeddings = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

c:\Users\vkfak\Documents\Projects\Bajaj_Hackrx\Pika\medibot_backend\mb_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set up LangChain's embedding model
from langchain.embeddings import SentenceTransformerEmbeddings

langchain_embeddings = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L6-v2"
)

In [ ]:



client = chromadb.Client(
    Settings(
        chroma_api_impl="rest",
        chroma_server_host="15.206.204.210",
        chroma_server_http_port="8000",
    )
)